# 1. Case Data Chunking - Embedding
## 1-1. 판례 

In [1]:
import os
import re
import glob
import pandas as pd
from tqdm.auto import tqdm
from langchain_ollama import ChatOllama
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

# LLM 설정 (제목 및 요약 생성용)
summary_llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1, num_predict=100)

c:\Users\Admin\Desktop\Project2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 통합 카테고리-키워드 가중치 사전
    법령, 판례, 상담 사례를 아우르는 포괄적 검색을 지원합니다.
    """
    return {
        # 1. 보증금_대항력: 보증금 보호 및 우선변제권 (최고 중요 리스크)
        '보증금_대항력': {
            '보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3,
            '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2,
            '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2,
            '반환보증': 2, 'HUG': 1, '보증보험': 2, '전세보증금': 2,
            '우선매수권': 3, '피해자결정': 2, '경매유예': 3,
            '임대보증금보증': 3, '일부보증': 2, '보증미가입': 3, '동의서': 2,
            '임차권이전': 2, '금융기관담보': 1, '임차권등기명령': 3, 
            '등기촉탁': 2, '기입등기': 2, '전자확정일자': 3,
        },

        # 2. 계약갱신_연장: 갱신요구권 및 실거주 거절 (빈번한 분쟁)
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '실거주': 3, '존속기간': 2, '2년': 1, '2+2': 2,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1, '직계존비속': 2
        },

        # 3. 수선_원상회복: 민법 제615조 기반의 유지보수 및 청소비 공제
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '원상에 회복': 3, '파손': 2, 
            '훼손': 2, '누수': 3, '곰팡이': 2, '보일러': 2, '필요비': 3, 
            '유익비': 3, '비용상환': 2, '보존행위': 2, '청소비': 3, 
            '원상복구': 2, '통상적 마멸': 3, '자연적 마멸': 2, '공제': 3,
            '공용부분': 2, '부속시설': 2, '교체': 1,
        },

        # 4. 생활환경_특약: 주거의 질과 관련된 커뮤니티 갈등 (신설)
        '생활환경_특약': {
            '층간소음': 3, '반려동물': 3, '애완동물': 3, '흡연': 3, '담배': 2,
            '주차': 2, '결로': 2, '방음': 2, '악취': 2, '공동생활': 2, 
            '금지사항': 2, '입주민갈등': 2, '벽지훼손': 2,
            '평온을 해치는 행위': 3, '관리규약': 2
        },

        # 5. 관리비_투명성: 관리비 꼼수 인상 및 내역 공개 (최신 이슈)
        '관리비_투명성': {
            '관리비': 3, '공공요금': 2, '정액관리비': 3, '세부내역': 2,
            '장기수선충당금': 3, '수선유지비': 2, '관리비인상': 2, '전기료': 1,
            '수도료': 1, '비용 청구': 1, '사용료': 2, '관리주체': 2, '내역공개': 2,
            '수선적립금': 3, '관리단': 2, '분담금': 2
        },

        # 6. 임대료_증감: 차임 증액 상한(5%) 및 월세 전환율
        '임대료_증감': {
            '차임': 3, '월세': 2, '증액': 3, '감액': 2, '인상': 2,
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1, '연체': 3, '3기': 3,
            '보증금전환': 3, '임차인동의': 2,
        },

        # 7. 권리_정보리스크: 신탁 부동산, 전세사기, 세금 체납 (정보 비대칭)
        '권리_정보리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '수탁자': 2, '근저당': 3, 
            '저당권': 3, '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, 
            '지방세': 2, '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '확정일자현황': 3, '조세채권': 2, '법정기일': 3, '미납국세열람': 2,
            '사용승인': 2, '무허가': 2, '거짓계약': 2,
            '법정기일': 3, '미납지방세열람': 3, '사해행위취소': 2,
            '본등기': 2, '직권말소': 2, '가등기': 2,
            '정보제공요청': 3, '열람신청': 2, '이해관계인 증명': 2,
        },

        # 8. 특약_유효성: 주임법 제10조(강행규정) 및 독소조항 탐지
        '특약_유효성': {
            '특약': 3, '특약사항': 3, '강행규정': 3, '제10조': 3,
            '효력이 없다': 3, '무효': 3, '불리한 약정': 3, '일방적 포기': 2,
            '편면적 강행규정': 2, '배제': 1, '표준임대차계약서': 3
        },

        # 9. 배상_책임: 민법 제390조/제750조 기반의 손해배상 및 위약금
        '손해배상_책임': {
            '손해배상': 3, '채무불이행': 3, '불법행위': 3, '위약금': 3,
            '과실': 2, '배상액': 2, '지연이자': 2, '이행지체': 2, '위반': 2
        },

        # 10. 계약해지_명도: 계약 종료 절차 및 퇴거/이사 관련
        '계약해지_명도': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 3, '이사': 1, '명도': 3, '합의해지': 2, 
            '통지': 1, '3개월': 2, '즉시해지': 3, '내용증명': 2,
            '임대차신고': 3, '거래신고': 3, '신고필증': 2, '등기사항': 2, '정정신청': 2,
        },

        # 11. 행정_절차: 확정일자 부여 및 전자계약 절차
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1
        },

        # 12. 분쟁해결_절차: 조정위원회 및 소송/경매 최후 수단
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 3, '지급명령': 3, '소송': 3,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2, '제소전화해': 3,
            '송달': 2, '강제집행': 3, '배당요구': 3, '매수신청보증': 2,
            '조정신청각하': 2, '출석요구': 1, '서면교부': 1,
            '소액사건': 3, '3,000만원': 3, '이행권고': 2, '공시최고': 1,
            '임차권등기명령신청': 3, '결정송달': 2, '신청서기재사항': 1,
        },

        # 13. 임차권_승계: 제9조 타겟팅 및 사실혼 가족 보호
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, 
            '공동상속': 2, '반환청구권': 2
        }
    }

def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters :
    - content : 분류할 텍스트 내용
    - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
    Returns:
    - 카테고리 리스트(점수 높은 순)"""
    category_keywords = get_law_category()
    category_scores = {}
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        # print(category, weighted_keywords)
        score = 0
        for keyword, weight in weighted_keywords.items():
            #if keyword in content: # content에 keyword 가 포함되어 있는지 여부
            #    score += weight
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0:
            category_scores[category] = score
    # print(category_scores)
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x:x[1], reverse=True) # 내림차순
    #print(sorted_categories)
    all_categories = [category[0] for category in sorted_categories]
    # print(all_categories)
    # 매칭되는 카테고리가 없으면 "기타" 반환
    if not all_categories:
        all_categories = ["기타"]
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories

In [7]:
def split_case_structurally(text, max_chars=2000):
    """
    판례를 [전문, 주문, 이유] 등 구조적으로 나누되, 
    각 섹션이 max_chars를 넘으면 추가로 분할합니다.
    """
    # 1. 주요 섹션 헤더 찾기 (공백/괄호 대응 강화)
    headers = [
        r'【\s*판\s*시\s*사\s*항\s*】', r'【\s*판\s*결\s*요\s*지\s*】', 
        r'【\s*전\s*문\s*】', r'【\s*주\s*문\s*】', 
        r'【\s*청\s*구\s*취\s*지\s*】', r'【\s*이\s*유\s*】'
    ]
    combined_header_regex = '|'.join(headers)
    
    # 헤더 위치 찾기
    matches = list(re.finditer(combined_header_regex, text))
    
    sections = []
    if not matches:
        # 헤더가 전혀 없으면 통으로 시작
        sections.append({"section": "본문", "content": text.strip()})
    else:
        # 첫 헤더 이전 내용(제목 등) 처리
        if matches[0].start() > 0:
            sections.append({"section": "서두", "content": text[:matches[0].start()].strip()})
            
        # 각 헤더 사이의 내용 추출
        for i in range(len(matches)):
            start = matches[i].start()
            end = matches[i+1].start() if i+1 < len(matches) else len(text)
            
            header_text = matches[i].group().strip()
            # 헤더 이름 깔끔하게 정리 (예: 【 이 유 】 -> 이유)
            section_name = re.sub(r'[【】\s]', '', header_text)
            content = text[start:end].strip()
            sections.append({"section": section_name, "content": content})

    # 2. 글자 수 제한(max_chars)에 따른 2차 분할 (Recursive Split)
    final_chunks = []
    for sec in sections:
        content = sec['content']
        if len(content) <= max_chars:
            final_chunks.append(sec)
        else:
            # max_chars 단위로 쪼개기
            num_parts = (len(content) // max_chars) + 1
            for p in range(num_parts):
                part_start = p * max_chars
                part_end = (p + 1) * max_chars
                chunk_part = content[part_start:part_end].strip()
                if chunk_part:
                    final_chunks.append({
                        "section": f"{sec['section']}_p{p+1}",
                        "content": chunk_part
                    })
                    
    return final_chunks

def extract_case_info(filename, content):
    """
    파일명과 본문에서 사건번호와 사건명을 추출합니다.
    """
    # 1. 사건번호 (파일명 우선: 2020나56247)
    case_no_match = re.search(r'\d{4}[가-힣]+\d+', filename)
    case_no = case_no_match.group() if case_no_match else "Unknown_Case"
    
    # 2. 사건명 (LLM 활용 추천 or 정규식)
    # 판례 본문 상단에 보통 '사건 2020나56247 건물인도' 형식으로 나옴
    title_match = re.search(r'사\s*건\s*\d{4}[가-힣]+\d+\s+(.*?)(\n|\r)', content)
    case_title = title_match.group(1).strip() if title_match else ""
    
    return case_no, case_title

In [ ]:
def process_single_case_file(file_path, priority=9):
    file_name = os.path.basename(file_path)
    
    # 1. 로드
    loader = Docx2txtLoader(file_path)
    docs = loader.load()
    full_text = docs[0].page_content
    
    # 2. 정보 추출
    case_no, case_name_raw = extract_case_info(file_name, full_text)
    
    # 3. 구조적 청킹
    structural_chunks = split_case_structurally(full_text, max_chars=2000)
    
    processed_docs = []
    for i, chunk_data in enumerate(structural_chunks):
        # chunk_id: PREC_2020나51962_000, 001, 002... 형태로 생성됨
        section = chunk_data['section']
        content = chunk_data['content']
        
        # 4. 카테고리 분류
        categories = categorize_content(content)
        
        # 5. LLM을 이용한 한 줄 요약 (Title 생성) - 중요!
        # 판례는 내용이 어려우므로 LLM이 "어떤 상황인지" 요약해주는 게 검색에 도움됨
        if i == 0: # 첫 번째 청크(보통 요지)일 때만 전체 요약 생성
            try:
                prompt = f"다음 판례 내용을 50자 이내의 검색용 제목으로 요약해줘 (예: 청소비 공제 특약의 유효성 판단).\n내용: {content[:1000]}"
                llm_response = summary_llm.invoke(prompt)
                ai_title = llm_response.content.strip()
            except:
                ai_title = case_name_raw
        else:
            ai_title = f"{case_name_raw} - {section}"

        # 6. 메타데이터 조립
        metadata = {
            "chunk_id": f"PREC_{case_no}_{i:03d}", # PREC: Precedent(판례) 접두어
            "priority": int(priority),  # 판례는 법령보다 낮은 우선순위(9)
            "category": categories,     # 리스트 형태
            "case_no": case_no,         # 별도 ID 지정
            "title": ai_title,          # LLM이 생성한 요약 제목
            "section": section,         # 판시사항/이유 구분
            "src_title": "판례",        # 출처 구분
            "source": file_name
        }
        
        doc = Document(page_content=content, metadata=metadata)
        processed_docs.append(doc)
        
    print(f"✅ 처리 완료: {case_no} ({len(processed_docs)} chunks)")
    return processed_docs

In [8]:
# 📂 처리할 폴더 경로 (변환된 .docx 파일이 있는 곳)
source_dir = r'C:\Users\Admin\Desktop\Project2\1_sourceData\1-1_sourceData\case\판례\docx' # 사용자 경로에 맞게 수정

# 1. .docx 파일 리스트업
target_files = glob.glob(os.path.join(source_dir, "*.docx"))

all_case_docs = []

print(f"🚀 총 {len(target_files)}개의 판례 파일을 처리합니다...")

for file_path in tqdm(target_files):
    # ~$: 임시 파일 제외
    if not os.path.basename(file_path).startswith("~$"):
        case_docs = process_single_case_file(file_path, priority=9) # 판례 Priority 9
        all_case_docs.extend(case_docs)

print(f"\n📦 판례 처리 완료! 총 {len(all_case_docs)}개의 청크가 준비되었습니다.")

# ==========================================
# [New] 최종 데이터 백업 (CSV 저장)
# ==========================================
import pandas as pd
import datetime

# 1. 데이터가 있는지 확인
if all_case_docs:
    print("📊 데이터프레임 변환 및 백업을 시작합니다...")
    
    # Document 객체 -> 리스트(Dictionary) 변환
    data_for_df = []
    for doc in all_case_docs:
        row = doc.metadata.copy()
        row['page_content'] = doc.page_content
        
        # [중요] 리스트 타입인 'category'는 CSV 저장을 위해 문자열로 변환
        if 'category' in row and isinstance(row['category'], list):
            row['category'] = ", ".join(row['category'])
            
        data_for_df.append(row)
    
    # 2. DataFrame 생성
    df_backup = pd.DataFrame(data_for_df)
    
    # 3. 컬럼 순서 보기 좋게 정렬 (선택 사항)
    # 주요 컬럼을 앞으로 배치
    cols = ['chunk_id', 'title', 'case_no', 'section', 'category', 'page_content', 'source']
    # 없는 컬럼은 무시하고 있는 것만 정렬, 나머지는 뒤로
    existing_cols = [c for c in cols if c in df_backup.columns]
    remaining_cols = [c for c in df_backup.columns if c not in cols]
    df_backup = df_backup[existing_cols + remaining_cols]

    # 4. CSV 파일로 저장 (파일명에 날짜/시간 포함)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
    backup_filename = f"processed_case_data_final_{timestamp}.csv"
    
    # utf-8-sig: 엑셀에서 한글 깨짐 방지
    df_backup.to_csv(backup_filename, index=False, encoding='utf-8-sig')
    
    print(f"✅ [백업 완료] 총 {len(df_backup)}개 행이 '{backup_filename}' 파일로 저장되었습니다.")
    
    # (선택) 상위 3개 행 미리보기
    display(df_backup.head(3)) 
else:
    print("⚠️ 저장할 데이터가 없습니다. 처리 과정(process_single_case_file)을 확인하세요.")

🚀 총 85개의 판례 파일을 처리합니다...


  1%|          | 1/85 [00:06<08:33,  6.11s/it]

✅ 처리 완료: 2000가단148507 (8 chunks)


  2%|▏         | 2/85 [00:10<07:06,  5.14s/it]

✅ 처리 완료: 2000다24184 (7 chunks)


...

 99%|█████████▉| 84/85 [04:53<00:03,  3.97s/it]

✅ 처리 완료: 2024다305087 (10 chunks)


100%|██████████| 85/85 [04:57<00:00,  3.51s/it]

✅ 처리 완료: 2024다326398 (8 chunks)

📦 판례 처리 완료! 총 693개의 청크가 준비되었습니다.
📊 데이터프레임 변환 및 백업을 시작합니다...
✅ [백업 완료] 총 693개 행이 'processed_case_data_final_20260122_1536.csv' 파일로 저장되었습니다.


,chunk_id,title,case_no,section,category,page_content,source,priority,src_title
0,PREC_2000가단148507_000,임대차보증금 관련 판결 요약: 서울지법 2001 판결 (보증금 처리 규정),2000가단148507,서두,보증금_대항력,"""임대차보증금"" (판례, 2000가단148507)\n\n임대차보증금\n\n[서울지법...",판례_2000가단148507.docx,9,판례
1,PREC_2000가단148507_001,- 판시사항,2000가단148507,판시사항,임차권_승계,【판시사항】\n\n주택임대차보호법상의 임차주택에 대한 임대인의 지위승계에 대하여 임...,판례_2000가단148507.docx,9,판례
2,PREC_2000가단148507_002,- 판결요지,2000가단148507,판결요지,"임차권_승계, 분쟁해결, 보증금_대항력","【판결요지】\n\n주거용건물의 임대차관계에서는 주택임대차보호법 제3조 제1항, 제2...",판례_2000가단148507.docx,9,판례


In [9]:
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from pinecone import Pinecone, ServerlessSpec

# 설정
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "case-index-final" # 판례/상담 전용 인덱스
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 인덱스 생성
if index_name not in pc.list_indexes().names():
    print(f"⚙️ '{index_name}' 인덱스를 생성합니다...")
    pc.create_index(
        name=index_name,
        dimension=4096,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# 업로드
print("🚀 Pinecone 업로드 시작...")
try:
    PineconeVectorStore.from_documents(
        documents=all_case_docs,
        embedding=embedding,
        index_name=index_name
    )
    print("🎉 판례 데이터 업로드 성공!")
except Exception as e:
    print(f"🔥 업로드 실패: {e}")

⚙️ 'case-index-final' 인덱스를 생성합니다...
🚀 Pinecone 업로드 시작...
🎉 판례 데이터 업로드 성공!


# 2. LLM Tuning

## 2-1. Normalizing Query Input - Langchain

In [11]:
# 주택임대차 챗봇 질문 표준화 사전
KEYWORD_DICT = {
    # 1. 계약 주체 및 대상
    "집주인": "임대인", "건물주": "임대인", "주인집": "임대인", "임대업자": "임대인", "새주인": "임대인",
    "세입자": "임차인", "월세입자": "임차인", "세들어사는사람": "임차인", "임차자": "임차인", "입주자": "임차인",
    "부동산": "공인중개사", "중개인": "공인중개사", "중개소": "공인중개사",
    "빌라": "임차주택", "아파트": "임차주택", "오피스텔": "임차주택", "우리집": "임차주택", "거주지": "임차주택",
    "계약서": "임대차계약증서", "집문서": "임대차계약증서", "종이": "임대차계약증서",

    # 2. 보증금 및 금전 (보증금_대항력, 임대료_증감)
    "전세금": "임차보증금", "보증금": "임차보증금", "맡긴돈": "임차보증금", "떼인돈": "임차보증금",
    "월세": "차임", "방세": "차임", "다달이내는지출": "차임", "렌트비": "차임", "임대료": "차임",
    "복비": "중개보수", "수수료": "중개보수", "중개비": "중개보수",
    "월세올리기": "차임증액", "인상": "증액", "더달라고함": "증액", "5프로": "5퍼센트상한",
    "월세깎기": "차임감액", "할인": "감액", "내리기": "감액",
    "돈먼저받기": "우선변제권", "순위": "우선변제권", "안전장치": "대항력", "돌려받기": "보증금반환",
    "보험": "반환보증", "허그": "HUG", "나라보증": "보증보험",

    # 3. 계약 상태 및 변화 (계약갱신, 계약해지_명도)
    "연장하기": "계약갱신요구권", "한번더살기": "계약갱신", "2플러스2": "계약갱신요구권", "갱신": "계약갱신",
    "재계약": "계약갱신", "자동연장": "묵시적갱신", "연락없음": "묵시적갱신", "그냥연장": "묵시적갱신",
    "이사": "주택의인도", "짐빼기": "주택의인도", "퇴거": "주택의인도", "방빼": "계약해지",
    "주소옮기기": "주민등록", "전입신고": "주민등록", "주소지이전": "주민등록",
    "집주인바뀜": "임대인지위승계", "주인바뀜": "임대인지위승계", "매매": "임대인지위승계",
    "나가라고함": "계약갱신거절", "쫓겨남": "명도", "비워달라": "명도", "중도해지": "계약해지",

    # 4. 수리 및 생활환경 (수선_원상회복, 생활환경_특약)
    "집고치기": "수선의무", "수리": "수선의무", "고쳐줘": "수선의무", "안고쳐줌": "수선의무위반",
    "곰팡이": "하자", "물샘": "누수", "보일러고장": "하자", "파손": "훼손",
    "깨끗이치우기": "원상회복의무", "원래대로해놓기": "원상회복", "청소비": "원상회복비용", "청소": "원상회복",
    "층간소음": "공동생활평온", "옆집소음": "방음", "개키우기": "반려동물특약", "담배": "흡연금지특약",

    # 5. 리스크 및 분쟁 (권리_정보리스크, 분쟁해결)
    "깡통전세": "전세피해", "사기": "전세사기", "경매넘어감": "권리리스크", "빚": "근저당",
    "세금안냄": "체납", "나라빚": "조세채권", "빌린돈": "가압류", "신탁": "신탁부동산",
    "특약": "특약사항", "불공정": "강행규정위반", "독소조항": "불리한약정", "효력있나": "무효여부",
    "조정위": "주택임대차분쟁조정위원회", "소송말고": "분쟁조정", "법원가기싫음": "분쟁조정",
    "집주인사망": "임차권승계", "자식상속": "임차권승계"
}

In [14]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 2. LLM 설정 (Exaone 3.5)
# 전처리는 창의성이 필요 없으므로 temperature=0으로 설정하여 일관성을 유지합니다.
response_llm = ChatOllama(model="exaone3.5:2.4b", temperature=0)

# 3. 프롬프트 템플릿 고도화
# 단순 치환뿐만 아니라 조사의 호응까지 맞추도록 지시합니다.
prompt = ChatPromptTemplate.from_template("""
당신은 법률 AI 챗봇의 전처리 담당자입니다. 
아래 [용어 사전]을 엄격히 준수하여 사용자의 질문을 '법률 표준어'로 변환해 주세요.

[수행 지침]
1. 사전에 있는 단어는 반드시 매핑된 법률 용어로 변경하세요.
2. 단어를 변경할 때 문맥에 맞게 조사(이/가, 을/를 등)나 서술어를 자연스럽게 수정하세요.
3. 사용자의 질문 의도를 왜곡하거나 추가적인 답변을 생성하지 마세요.
4. 오직 '변경된 질문' 텍스트만 출력하세요. (설명 금지)

[용어 사전]
{dictionary}

사용자 질문: {question}
변경된 질문:""")

# 4. 체인 생성
keyword_chain = prompt | response_llm | StrOutputParser()

# 5. 실행 함수
def normalize_query(user_query):
    """
    KEYWORD_DICT를 사용하여 사용자 쿼리를 법률 용어로 표준화합니다.
    """
    try:
        # invoke 할 때 dictionary에 딕셔너리 객체(KEYWORD_DICT)를 그대로 넘깁니다.
        normalized = keyword_chain.invoke({
            "dictionary": KEYWORD_DICT, 
            "question": user_query
        })
        return normalized.strip()
    except Exception as e:
        print(f"⚠️ 전처리 에러: {e}")
        return user_query

# ==========================================
# 6. 테스트
# ==========================================
query = "집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?"
normalized_query = normalize_query(query)

print(f"원본: {query}")
print(f"변환: {normalized_query}")

원본: 집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?
변환: 임차인이 임대인에게 월세 인상 요구에 대해 거부 의사를 밝혔으나, 임대인이 계약 갱신 요구 없이 다음 달까지 퇴거를 요구하고 있습니다. 임대차계약증서에는 이러한 구체적인 인상 내용이 명시되지 않았는데, 이러한 상황에서 실제로 임차인이 쫓겨나야 하는지 우려되시나요?


## 2-2. RAG 조회 함수 정의

In [19]:
import os
import time
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

# 1. 환경 설정
load_dotenv(override=True)
pc_api_key = os.getenv("PINECONE_API_KEY")
up_api_key = os.getenv("UPSTAGE_API_KEY")
cohere_api_key = os.getenv("COHERE_API_KEY") # Reranking을 위해 필수 권장

# 2. Pinecone & Embedding 초기화
pc = Pinecone(api_key=pc_api_key)
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 3. Triple VectorStore 연결 (법률, 규칙, 판례)
print("🔗 3중 인덱스 연결 시도...")

# (1) Law Index: 주임법, 민법 등 핵심 법률 (Priority 1,2,4,5)
law_store = PineconeVectorStore(
    index_name="law-index-final",
    embedding=embedding,
    pinecone_api_key=pc_api_key
)

# (2) Rule Index: 시행규칙, 조례, 절차 등 (Priority 3,6,7,8,11)
rule_store = PineconeVectorStore(
    index_name="rule-index-final",
    embedding=embedding,
    pinecone_api_key=pc_api_key
)

# (3) Case Index: 판례, 상담사례 (Priority 9)
case_store = PineconeVectorStore(
    index_name="case-index-final",
    embedding=embedding,
    pinecone_api_key=pc_api_key
)

print("✅ [Law / Rule / Case] 3개 인덱스 로드 완료!")

🔗 3중 인덱스 연결 시도...
✅ [Law / Rule / Case] 3개 인덱스 로드 완료!


In [20]:
def get_full_case_context(case_no, case_index, top_k=50):
    """
    특정 사건번호(case_no)를 가진 모든 청크를 가져와서 판례 전문을 재구성합니다.
    """
    try:
        # 더미 쿼리 사용으로 API 에러 방지
        results = case_index.similarity_search(
            query="판례 전문 검색", 
            k=top_k, 
            filter={"case_no": {"$eq": case_no}}
        )
        
        # chunk_id 순 정렬
        sorted_docs = sorted(results, key=lambda x: x.metadata.get('chunk_id', ''))
        
        # 중복 제거 및 병합
        seen_chunks = set()
        unique_docs = []
        for doc in sorted_docs:
            cid = doc.metadata.get('chunk_id')
            if cid and cid not in seen_chunks:
                unique_docs.append(doc)
                seen_chunks.add(cid)
        
        full_text = "\n".join([doc.page_content for doc in unique_docs])
        return full_text
        
    except Exception as e:
        print(f"⚠️ 판례 전문 로딩 실패 ({case_no}): {e}")
        return ""

In [23]:
import cohere

def triple_hybrid_retrieval(query, law_store, rule_store, case_store, k_law=5, k_rule=5, k_case=3, score_threshold=0.2):
    """
    1단계: Law, Rule, Case 인덱스에서 관련 문서 수집
    2단계: Rerank로 관련도 높은 문서 선별
    3단계: Priority 메타데이터 기준으로 '법적 위계' 정렬하여 반환
    """
    print(f"🔍 [통합 검색] 쿼리: '{query}'")
    
    # 1. 병렬 검색 (Parallel Retrieval)
    # (A) Law: 법적 근거 (예: 주임법 제3조)
    docs_law = law_store.similarity_search(query, k=k_law * 2)
    
    # (B) Rule: 행정 절차 및 서식 (예: 확정일자 부여 규칙)
    docs_rule = rule_store.similarity_search(query, k=k_rule * 2)
    
    # (C) Case: 유사 판례 (예: 대법원 2020다...)
    docs_case_initial = case_store.similarity_search(query, k=k_case * 2)
    
    # 2. 판례 문맥 확장 (Context Expansion)
    docs_case_expanded = []
    seen_cases = set()
    
    for doc in docs_case_initial:
        case_no = doc.metadata.get('case_no')
        if case_no and case_no not in seen_cases:
            full_text = get_full_case_context(case_no, case_store)
            if full_text:
                # 판례 전문으로 교체하되, 출처 표기를 위해 메타데이터 유지
                new_doc = doc 
                new_doc.page_content = f"[판례 전문: {doc.metadata.get('title')}]\n{full_text}"
                docs_case_expanded.append(new_doc)
                seen_cases.add(case_no)
            
            if len(docs_case_expanded) >= k_case:
                break
    
    # 3. 문서 통합 (Law + Rule + Case)
    combined_docs = docs_law + docs_rule + docs_case_expanded
    
    # 4. Reranking (중요: 서로 다른 인덱스의 점수를 보정하기 위함)
    selected_docs = combined_docs # 기본값

    if cohere_api_key:
        try:
            co = cohere.Client(api_key=cohere_api_key)
            docs_content = [d.page_content for d in combined_docs]
            
            # 한국어에 특화된 다국어 모델 사용
            rerank_results = co.rerank(
                model="rerank-multilingual-v3.0",
                query=query,
                documents=docs_content,
                top_n=len(combined_docs) 
            )
            
            filtered_docs = []
            print(f"📊 Rerank 결과 (총 {len(combined_docs)}개 중 선별):")
            print(f"📊 Rerank 관련도 점수 (Threshold {score_threshold}):")
            for r in rerank_results.results:
                # 관련도 점수가 너무 낮은 것은 제외 (Noise Filtering)
                if r.relevance_score > score_threshold: 
                    doc = combined_docs[r.index]
                    # 디버깅용 출력
                    p = doc.metadata.get('priority', 99)
                    t = doc.metadata.get('title', 'Untitled')
                    print(f" - [Score: {r.relevance_score:.4f}] [P-{p}] {t}")
                    filtered_docs.append(doc)
            selected_docs = filtered_docs
            
        except Exception as e:
            print(f"⚠️ Rerank 실패 (기본 병합 반환): {e}")
            return combined_docs

    # 4. Priority Sorting (법적 권위 정렬)
    # priority 숫자 오름차순(1→9)으로 정렬
    # priority가 없는 경우 99로 취급하여 맨 뒤로
    sorted_docs = sorted(selected_docs, key=lambda x: int(x.metadata.get('priority', 99)))
    return sorted_docs

## 2-3. LLM Pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

def format_context_with_hierarchy(docs):
    """
    문서들을 Priority에 따라 그룹화하여 문자열로 반환.
    """
    section_1_law = []   # Priority 1, 2, 4, 5 (법률, 시행령)
    section_2_rule = []  # Priority 3, 6, 7, 8, 11 (규칙, 조례)
    section_3_case = []  # Priority 9 (판례, 해석)
    
    for doc in docs:
        p = int(doc.metadata.get('priority', 99))
        src = doc.metadata.get('src_title', '자료')
        title = doc.metadata.get('title', '')
        content = doc.page_content
        
        entry = f"[{src}] {title}\n{content}"
        
        if p in [1, 2, 4, 5]:
            section_1_law.append(entry)
        elif p in [3, 6, 7, 8, 11]:
            section_2_rule.append(entry)
        else:
            section_3_case.append(entry)
            
    # LLM이 읽을 최종 컨텍스트 조립
    formatted_text = ""
    
    if section_1_law:
        formatted_text += "## [SECTION 1: 핵심 법령 (최우선 법적 근거)]\n" + "\n\n".join(section_1_law) + "\n\n"
    if section_2_rule:
        formatted_text += "## [SECTION 2: 관련 규정 및 절차 (세부 기준)]\n" + "\n\n".join(section_2_rule) + "\n\n"
    if section_3_case:
        formatted_text += "## [SECTION 3: 판례 및 해석 사례 (적용 예시)]\n" + "\n\n".join(section_3_case) + "\n\n"
        
    return formatted_text

def generate_final_answer(user_input):
    # 1. 질문 표준화
    try:
        normalized_query = normalize_query(user_input)
        print(f"🔄 표준화된 질문: {normalized_query}")
    except:
        normalized_query = user_input
    
    # 2. 통합 검색 및 위계 정렬
    retrieved_docs = triple_hybrid_retrieval(
        normalized_query, 
        law_store, rule_store, case_store,
        k_law=3, k_rule=3, k_case=2
    )
    
    if not retrieved_docs:
        return "죄송합니다. 관련 법령이나 판례를 찾을 수 없습니다."

    # 3. 위계 구조화된 컨텍스트 생성
    hierarchical_context = format_context_with_hierarchy(retrieved_docs)

    # 4. LLM 프롬프트 (위계 구조 반영)
    system_prompt = """
    당신은 대한민국 '주택 전월세 사기 예방 및 임대차 법률 전문가 AI'입니다.
    사용자의 질문에 대해 제공된 [법적 위계가 정리된 참고 문서]를 바탕으로 답변하세요.

    [답변 생성 원칙]
    1. **법적 위계 준수**: 
       - 반드시 [SECTION 1: 핵심 법령]의 내용을 최우선 판단 기준으로 삼으세요.
       - [SECTION 1]의 내용이 모호할 때만 [SECTION 2]와 [SECTION 3]를 보충 근거로 활용하세요.
       - 만약 [SECTION 3: 판례]가 [SECTION 1: 법령]과 다르게 해석되는 특수한 경우라면, "원칙은 법령에 따르나, 판례는 예외적으로..."라고 설명하세요.
    
    2. **답변 구조**:
       - **핵심 결론**: 질문에 대한 결론을 두괄식으로 요약.
       - **법적 근거**: "주택임대차보호법 제O조에 따르면..." (SECTION 1 인용)
       - **실무 절차**: 필요시 신고 방법, 서류 등 안내 (SECTION 2 인용)
       - **참고 사례**: 유사한 상황에서의 판결이나 해석 (SECTION 3 인용)
       
    3. **주의사항**:
       - 사용자의 계약서 내용이 법령(강행규정)에 위반되면 "효력이 없다(무효)"고 명확히 경고하세요.
       - 법률적 조언일 뿐이므로, 최종적으로는 변호사 등의 전문가 확인이 필요함을 고지하세요.

    [법적 위계가 정리된 참고 문서]
    {context}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{question}"),
    ])
    
    llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)
    chain = prompt | llm | StrOutputParser()
    
    print("🤖 답변 생성 중...")
    return chain.invoke({"context": hierarchical_context, "question": normalized_query})

# 3. LLM Test

In [26]:
test_query = "확정일자 받는 수수료가 얼마야? 그리고 집주인이 실거주한다고 나가라는데 진짜인지 어떻게 확인해?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 확정일자 받는 데 드는 중개보수는 얼마인가요? 그리고 임대인이 실제로 거주한다고 주장하며 퇴거를 요구할 때 그 주장의 진위를 어떻게 확인할 수 있나요?
🔍 [통합 검색] 쿼리: '확정일자 받는 데 드는 중개보수는 얼마인가요? 그리고 임대인이 실제로 거주한다고 주장하며 퇴거를 요구할 때 그 주장의 진위를 어떻게 확인할 수 있나요?'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.5546] [P-3.0] 제13조 수수료 규정
🤖 답변 생성 중...
### 확정일자 부여 중개보수

**핵심 결론**: 확정일자 부여 수수료는 주로 임대차 계약서의 종류와 제공 방식에 따라 달라집니다. 일반적으로 중개보수는 다음과 같이 적용됩니다:

- **서면 확정일자 부여**: 각 계약서당 600원이 부과되며, 4장을 초과하는 경우 추가로 각 1장당 100원이 부과됩니다.
- **인터넷 등기소를 통한 전자 확정일자 부여**: 각 신청당 500원이 부과되며, 초과 수수료는 적용되지 않습니다.

**법적 근거**: 주택임대차보호법 제13조에 따르면 확정일자 부여 수수료는 계약서의 수와 방식에 따라 다르게 규정되어 있습니다. 특히, 인터넷 등기소를 이용한 경우에는 수수료가 상대적으로 낮게 책정되어 있습니다.

### 임대인 거주 주장 진위 확인 방법

**핵심 결론**: 임대인의 거주 주장 진위를 확인하는 방법은 다음과 같습니다:

1. **증빙 자료 수집**:
   - **임대차 계약서**: 계약서에 임대인의 주소나 연락처가 명시되어 있는지 확인합니다.
   - **주민등록등본**: 임대인의 실제 거주 주소를 확인할 수 있습니다.
   - **통신 요금 청구서**: 임대인의 전화번호나 인터넷 서비스 이용 내역을 통해 거주 여부를 간접적으로 확인할 수 있습니다.
   - **사진 및 동영상**: 임대인이 실제로 거주하는 공간의 사진이나 동영상을 촬영하여 비교합니다.

2. **현장 조사*

In [27]:
test_query = "집주인이 특약으로 '1년 뒤 퇴거'를 넣자는데, 법적으로 효력이 있어? 판례는 어때?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임차인이 '1년 뒤 퇴거' 특약을 임대차계약증서에 기재했을 때, 법적으로 그 효력은 어떻게 될까요? 관련 판례는 어떠한가요?
🔍 [통합 검색] 쿼리: '임차인이 '1년 뒤 퇴거' 특약을 임대차계약증서에 기재했을 때, 법적으로 그 효력은 어떻게 될까요? 관련 판례는 어떠한가요?'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.7254] [P-9.0]  - 이유
🤖 답변 생성 중...
### 핵심 결론
임차인이 임대차계약증서에 "1년 뒤 퇴거" 특약을 명시적으로 기재하였다면, 이 특약은 **법적으로 유효**하며, 임대인과 임차인 간의 합의에 따라 계약 기간이 종료된 후 일정 기간 동안 임차인의 퇴거 의무가 인정됩니다. 그러나 이 특약이 법적으로 강제력을 가지려면 다음과 같은 조건들이 충족되어야 합니다:

- **명확성**: 특약이 명확하게 기재되어 있어야 합니다.
- **합의**: 양 당사자 간에 합의가 이루어져야 합니다.
- **법률 준수**: 특약이 주택임대차보호법 등 관련 법령과 충돌하지 않아야 합니다.

### 법적 근거
- **주택임대차보호법 제6조**: 임대차계약의 갱신거절 통지 기간과 관련된 규정을 명시하고 있습니다. 하지만 특약이 명시적으로 기재되어 있다면, 이 특약이 법적으로 유효하다는 점을 강조할 수 있습니다.
  - **주택임대차보호법 제6조 제1항**: 임대인이 임대차기간의 끝나기 6개월 전부터 2개월 전까지의 기간에 임차인에게 갱신거절의 통지를 하지 않으면 계약이 자동으로 갱신된 것으로 간주됩니다.
  - **주택임대차보호법 제6조 제2항**: 임대인이 임차인의 계약갱신 요구를 정당한 사유 없이 거절할 수 없습니다.

### 실무 절차
임차인이 "1년 뒤 퇴거" 특약을 임대차계약증서에 기재하였다면, 다음과 같은 절차를 통해 특약의 효력을 확인하고 이행할 수 있습니다:

1. **서면 확인**: 특약이 명확하게 기재되어 있는지 확인합니다. 계

In [28]:
test_query = "세입자가 월세를 3번 밀렸는데, 계약 해지하고 보증금에서 까도 되나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임차인이 월세를 세 차례 연체한 경우, 계약 해지가 가능하며, 보증금에서 차감할 수 있는지에 대한 확인이 필요합니다.
🔍 [통합 검색] 쿼리: '임차인이 월세를 세 차례 연체한 경우, 계약 해지가 가능하며, 보증금에서 차감할 수 있는지에 대한 확인이 필요합니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.6905] [P-9.0]  - 이유_p1
 - [Score: 0.5365] [P-4.0] 제640조 차임연체와 해지
🤖 답변 생성 중...
임차인이 월세를 세 차례 연체한 경우, 계약 해지와 보증금 관련 처리는 다음과 같은 절차와 고려사항을 따르는 것이 일반적입니다:

### 1. **계약 조건 확인**
   - **계약서 검토**: 먼저 해당 임대차 계약서를 철저히 검토해야 합니다. 계약서에 명시된 연체 기간, 해지 조건, 보증금 반환 절차 등이 중요합니다.
   - **연체 횟수 기준**: 계약서에 명시된 연체 횟수 기준이 몇 번인지 확인해야 합니다. 세 차례 연체가 해당 기준에 부합하는지 확인하세요.

### 2. **해지 절차**
   - **해지 통지**: 임대인은 일반적으로 서면으로 해지 통지를 해야 합니다. 통지 내용에는 해지 사유(연체 횟수), 해지 예정일 등이 포함되어야 합니다.
   - **서면 통지**: 이메일, 우편 등 공식적인 방법으로 통지해야 합니다. 이는 법적 효력을 갖추기 위함입니다.

### 3. **보증금 처리**
   - **보증금 반환 조건**: 계약서에 따라 보증금 반환 조건이 다를 수 있습니다. 일반적으로 다음과 같은 경우가 많습니다:
     - **정상 해지**: 계약이 정상적으로 해지되었을 때 보증금이 반환됩니다.
     - **연체 후 해지**: 세 차례 이상 연체 후 해지된 경우에도 보증금 반환 조건이 명시되어 있을 것입니다. 계약서에 따라:
       - **잔여 보증금 반환**: 연체 기간 동안의 보

In [29]:
test_query = "집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임차인이 임대인에게 월세 인상 요구에 대해 거부 의사를 밝혔으나, 임대인이 계약 갱신 요구 없이 다음 달까지 퇴거를 요구하고 있습니다. 임대차계약증서에는 이러한 구체적인 인상 내용이 명시되지 않았는데, 이러한 상황에서 실제로 임차인이 쫓겨나야 하는지 우려되시나요?
🔍 [통합 검색] 쿼리: '임차인이 임대인에게 월세 인상 요구에 대해 거부 의사를 밝혔으나, 임대인이 계약 갱신 요구 없이 다음 달까지 퇴거를 요구하고 있습니다. 임대차계약증서에는 이러한 구체적인 인상 내용이 명시되지 않았는데, 이러한 상황에서 실제로 임차인이 쫓겨나야 하는지 우려되시나요?'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.9338] [P-1.0] 제6조 계약갱신 요구권 제한 및 손해배상 규정
 - [Score: 0.9075] [P-9.0]  - 이유_p1
 - [Score: 0.8759] [P-1.0] 제6조 계약 갱신 규정
 - [Score: 0.8013] [P-9.0]  - 이유_p1
 - [Score: 0.7666] [P-3.0] 제6조 주택 임대차 계약 신고 의무화
 - [Score: 0.6604] [P-1.0] 제10조 초과 차임 반환 청구
 - [Score: 0.6302] [P-2.0] 제8조 차임 증액 제한 기준
 - [Score: 0.6222] [P-1.0] 제7조 차임 증감 청구권
 - [Score: 0.6222] [P-3.0] 제3조 확정일자 부여 시 확인사항
 - [Score: 0.6214] [P-1.0] 제6조 묵시적 갱신 시 계약 해지 규정
 - [Score: 0.5315] [P-6.0] 제18조 임대보증금과 월임대료 전환 규정
 - [Score: 0.4728] [P-6.0] 제2조 임차권등기명령신청서 기재사항
 - [Score: 0.3911] [P-3.0] 제7조 임대차 정보제공 요청 방법
 - [Score: 0.3526] [P-6.0] 제3조 임차권등기명령신

In [30]:
test_query = "전입신고랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임차인으로서 전입신고와 확정일자를 완료하셨다면, 확정일자부에 기재된 내용이 정확한지 확인하는 것이 중요합니다. 특히 임대차 계약의 조건, 보증금 액수, 차임 금액 등이 명확히 기재되어 있는지 살펴보는 것이 좋습니다. 이러한 정보는 향후 계약 관련 분쟁이나 문제 발생 시 근거 자료로 활용될 수 있으므로, 내용의 정확성은 매우 중요합니다.
🔍 [통합 검색] 쿼리: '임차인으로서 전입신고와 확정일자를 완료하셨다면, 확정일자부에 기재된 내용이 정확한지 확인하는 것이 중요합니다. 특히 임대차 계약의 조건, 보증금 액수, 차임 금액 등이 명확히 기재되어 있는지 살펴보는 것이 좋습니다. 이러한 정보는 향후 계약 관련 분쟁이나 문제 발생 시 근거 자료로 활용될 수 있으므로, 내용의 정확성은 매우 중요합니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.9240] [P-9.0]  - 이유
 - [Score: 0.9208] [P-9.0]  - 판결요지
 - [Score: 0.9205] [P-1.0] 제3조 전세사기피해자 요건
 - [Score: 0.9014] [P-3.0] 제3조 확정일자 부여 시 확인사항
 - [Score: 0.8694] [P-3.0] 제9조 전자확정일자 정보 작성 규정
 - [Score: 0.8379] [P-1.0] 제3조 임대차 확정일자 부여 및 정보제공 관련 규정
 - [Score: 0.5038] [P-1.0] 제3조 주택임대차 등기 효력 관련 규정
 - [Score: 0.4907] [P-3.0] 제11조 확정일자 정보제공 요청 방법
 - [Score: 0.4660] [P-6.0] 제6조 임차권등기 기록사항 규정
 - [Score: 0.3940] [P-1.0] 제6조 주택 임대차 계약 변경 및 해제 신고
 - [Score: 0.3876] [P-1.0] 제3조 임대인 정보 제시 의무
 - [Score: 0.3662] [P-1.0] 제6조 주택 임대차 계

In [31]:
test_query = "계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임대차계약증서에 1년 거주 기간을 명시했다면, 계약 기간이 만료되면 임대인이 무조건 계약을 해지할 수 있는 건가요?
🔍 [통합 검색] 쿼리: '임대차계약증서에 1년 거주 기간을 명시했다면, 계약 기간이 만료되면 임대인이 무조건 계약을 해지할 수 있는 건가요?'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.7194] [P-9.0]  - 이유_p1
 - [Score: 0.6678] [P-9.0]  - 이유_p1
 - [Score: 0.2726] [P-1.0] 제4조 임대차기간 규정
🤖 답변 생성 중...
임대차 계약에서 계약 기간이 명시되어 있더라도, 임대인이 무조건적으로 계약을 해지할 수 있는 것은 아닙니다. 일반적으로 다음과 같은 요소들이 고려됩니다:

1. **계약 조건**: 계약서에 명시된 조건에 따라 달라집니다. 대부분의 경우, 계약 기간이 만료되면 자동으로 계약이 종료되지만, 임대인과 임차인 간에 합의가 있으면 연장 계약을 체결할 수 있습니다.

2. **임대인의 동의**: 계약 기간 만료 후에도 임대인이 임차인에게 재계약을 제안하거나 동의할 수 있습니다. 이 경우 임차인은 임대인의 제안을 수락하거나 거절할 수 있습니다.

3. **법적 요건**: 일부 지역의 법률이나 규정에 따라 계약 기간 만료 후에도 임대인이 강제로 계약을 해지할 수 있는 제한이 있을 수 있습니다. 예를 들어, 특정 기간 동안 임차인이 임대료를 지불하지 않거나 계약 조건을 위반한 경우 임대인은 법적 절차를 통해 계약을 해지할 수 있지만, 이는 계약 기간 만료 자체보다는 위반 사항에 따른 것입니다.

따라서, 계약 기간이 만료되었다고 해서 임대인이 무조건적으로 계약을 해지할 수 있는 것은 아니며, 상황에 따라 임대인과 임차인 간의 합의나 법적 요건에 따라 달라질 수 있습니다. 구체적인 상황에 대해서는 해당 지역의 법률 자문을 받는 것이 가장 안전한 방법입니다.


In [32]:
test_query = "제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 집주인이랑 말로만 얘기했어요. 문제 될 수 있나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 제가 임차주택에 대해 임차인이 일시적으로 임대인의 동의 하에 다른 사람이 거주하도록 허용받았는지 명확히 확인해야 합니다. 임대차계약증서에 명시되지 않은 상황 변경은 임대인의 동의 없이는 계약에 영향을 줄 수 있으므로, 계약갱신요구권이나 계약해지와 관련된 조항을 검토하고, 필요하다면 공인중개사와 상의하여 법적 문제 발생 가능성을 확인하는 것이 바람직합니다.
🔍 [통합 검색] 쿼리: '제가 임차주택에 대해 임차인이 일시적으로 임대인의 동의 하에 다른 사람이 거주하도록 허용받았는지 명확히 확인해야 합니다. 임대차계약증서에 명시되지 않은 상황 변경은 임대인의 동의 없이는 계약에 영향을 줄 수 있으므로, 계약갱신요구권이나 계약해지와 관련된 조항을 검토하고, 필요하다면 공인중개사와 상의하여 법적 문제 발생 가능성을 확인하는 것이 바람직합니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.7096] [P-9.0]  - 이유_p3
 - [Score: 0.5782] [P-6.0] 제2조 임차권등기명령신청서 기재사항
 - [Score: 0.5681] [P-3.0] 제6조 주택 임대차 신고 절차
 - [Score: 0.5403] [P-9.0]  - 이유_p2
 - [Score: 0.4754] [P-4.0] 제629조 임차권 양도 및 전대 제한
 - [Score: 0.4551] [P-4.0] 제631조 전차인 권리의 존속
 - [Score: 0.4095] [P-3.0] 제12조 임대차 정보 열람 및 제공 규정
 - [Score: 0.3924] [P-1.0] 제6조 계약갱신 요구권 제한 및 손해배상 규정
 - [Score: 0.3223] [P-4.0] 제639조 묵시적 갱신
 - [Score: 0.2672] [P-1.0] 제6조 묵시적 갱신 시 계약 해지 규정
 - [Score: 0.2482] [P-3.0] 제7조 임대차 정보제공 요청 방법
 - [Score: 0.2048] [P

In [33]:
test_query = "계약 끝날 때까지 집주인이 아무 말 안 했는데, 이거 자동으로 연장된 건가요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 계약 끝날 때까지 임대인의 언급이 없었다면, 묵시적 계약 갱신이 이루어졌을 가능성이 있습니다.
🔍 [통합 검색] 쿼리: '계약 끝날 때까지 임대인의 언급이 없었다면, 묵시적 계약 갱신이 이루어졌을 가능성이 있습니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.8774] [P-9.0]  - 이유_p2
 - [Score: 0.8353] [P-9.0]  - 이유_p2
 - [Score: 0.5811] [P-1.0] 제6조 계약 갱신 규정
 - [Score: 0.2163] [P-1.0] 제6조 묵시적 갱신 시 계약 해지 규정
🤖 답변 생성 중...
네, 맞습니다. 한국의 민법에서는 특정 조건 하에 묵시적 계약 갱신에 대한 규정이 있습니다. 주요 내용은 다음과 같습니다:

1. **묵시적 계약 갱신**: 계약이 끝날 때까지 임대인의 명시적인 거절이나 갱신 거부의 언급이 없다면, 기존 계약이 자동으로 갱신되는 것을 의미합니다. 특히 다음과 같은 상황에서 묵시적 갱신이 이루어질 가능성이 큽니다:
   - **임대인의 행동**: 임대인이 임대료를 지불하지 않거나, 계약 조건을 준수하지 않는 등의 행동이 있었음에도 불구하고, 임차인이 계속해서 거주하고 임대료를 지불하는 경우.
   - **임대인의 언급 부재**: 계약 종료 시점에서 임대인이 갱신 거부를 명시적으로 언급하지 않은 경우.

2. **법적 근거**: 민법 제628조에 따르면, 계약 기간이 만료되었음에도 불구하고 임대인이 임차인에게 갱신 거부 의사를 명확히 밝히지 않았다면, 임차인은 계약을 갱신할 권리가 있습니다. 하지만 이는 임대인의 의사가 중요한 요소로 작용합니다.

따라서, 계약 종료 시점에서 임대인의 언급이 없었다면, 묵시적 계약 갱신이 이루어질 가능성이 상당히 높다고 볼 수 있습니다. 하지만 구체적인 상황과 관련 법률의 적용 여부는 해당 지역의 법률 자문을 받는 것이 가장 확실한 방법입니다.


In [34]:
test_query = "계약이 그냥 연장된 줄 모르고 살고 있었는데요. 제가 갑자기 이사 가야 하면 언제까지 살아야 하나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임차인으로서 묵시적 갱신 기간 내에 거주하고 계시다면, 계약이 자동으로 연장된 상태입니다. 계약 갱신 요구권이 있는 경우, 임대인에게 계약 갱신을 요구할 수 있습니다. 만약 계약이 연장되었다면, 현재 계약 기간이 만료되기 전까지 거주해야 합니다. 구체적인 만료일은 계약서에 명시되어 있으므로 확인이 필요합니다. 만약 갱신 요구를 하지 않았다면, 계약 기간이 만료되는 날까지 거주하시면 됩니다.
🔍 [통합 검색] 쿼리: '임차인으로서 묵시적 갱신 기간 내에 거주하고 계시다면, 계약이 자동으로 연장된 상태입니다. 계약 갱신 요구권이 있는 경우, 임대인에게 계약 갱신을 요구할 수 있습니다. 만약 계약이 연장되었다면, 현재 계약 기간이 만료되기 전까지 거주해야 합니다. 구체적인 만료일은 계약서에 명시되어 있으므로 확인이 필요합니다. 만약 갱신 요구를 하지 않았다면, 계약 기간이 만료되는 날까지 거주하시면 됩니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.9541] [P-1.0] 제6조 묵시적 갱신 시 계약 해지 규정
 - [Score: 0.8897] [P-4.0] 제639조 묵시적 갱신
 - [Score: 0.8084] [P-9.0]  - 이유_p3
 - [Score: 0.3175] [P-3.0] 제3조 확정일자 부여 시 확인사항
 - [Score: 0.3014] [P-9.0]  - 판시사항
 - [Score: 0.2791] [P-1.0] 제6조 계약 갱신 규정
 - [Score: 0.2672] [P-1.0] 제4조 임대차기간 규정
 - [Score: 0.2285] [P-3.0] 제12조 임대차 정보 열람 및 제공 규정
 - [Score: 0.2217] [P-3.0] 제7조 임대차 정보제공 요청 방법
 - [Score: 0.2172] [P-1.0] 제6조 계약갱신 요구권 제한 및 손해배상 규정
 - [Score: 0.2103] [P-6.0] 제3조 임차권등기명령신

In [35]:
test_query = "집주인이 실거주한다고 갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임차인이 임대인으로부터 계약갱신 요구 거절을 받았는데, 실제로 거주하지 않는 경우, 임대차 계약 관련 기록을 확인할 수 있나요?
🔍 [통합 검색] 쿼리: '임차인이 임대인으로부터 계약갱신 요구 거절을 받았는데, 실제로 거주하지 않는 경우, 임대차 계약 관련 기록을 확인할 수 있나요?'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.9970] [P-9.0]  - 이유_p1
 - [Score: 0.9910] [P-9.0]  - 판시사항
 - [Score: 0.9591] [P-1.0] 제6조 계약갱신 요구권 제한 및 손해배상 규정
 - [Score: 0.3443] [P-3.0] 제6조 주택 임대차 계약 신고 의무화
 - [Score: 0.3048] [P-1.0] 제6조 계약 갱신 규정
🤖 답변 생성 중...
임차인이 임대인으로부터 계약갱신 요구 거절을 받았음에도 불구하고 실제로 거주하지 않는 상황에서 임대차 계약 관련 기록을 확인하는 것은 가능하지만, 그 과정과 방법은 다음과 같습니다:

1. **임대차 계약서 확인**:
   - **원본 계약서**: 가장 확실한 방법은 원본 임대차 계약서를 확인하는 것입니다. 계약서에는 계약 기간, 임대료, 거주 조건 등이 명시되어 있어 임대인의 요구와 실제 상황을 비교할 수 있습니다.
   - **변경 사항**: 계약서에 변경 사항이 기록되어 있다면, 그 내용을 통해 임대인의 요구와 임차인의 실제 상황을 분석할 수 있습니다.

2. **임대차 관련 문서**:
   - **임대료 납부 내역**: 은행 계좌 내역이나 카드 사용 내역 등을 통해 임대료를 실제로 납부했는지 확인할 수 있습니다.
   - **세금 및 공과금 납부 내역**: 주민등록번호나 사업자등록번호를 통해 해당 기간 동안의 세금 및 공과금 납부 내역을 확인할 수 있습니다.

3. **공인중개사 또는 법률 자문**:
   - **전문가 상담**: 부동산 중개업자나 법률 

In [36]:
test_query = "재계약하면서 월세를 한 번에 많이 올리자고 하는데, 집주인이 정하는 대로 따라야 하나요? 그리고 월세를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 한 번만 올릴 수 있다는 게 진짜인가요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임대차계약증서에 명시된 증액 비율을 초과하여 월세를 인상하려는 경우, 임차인은 임대인의 결정에 무조건 따르지 않아도 됩니다. 또한, 계약 기간 내에서 월세를 인상하는 횟수에 대한 규정은 지역 법률과 계약서 내용에 따라 다를 수 있으나, 일반적으로 1년에 한 번만 인상할 수 있다는 것은 상한선일 수 있습니다. 구체적인 조항 확인을 위해 임대차계약증서를 참조하거나 법률 전문가의 조언을 구하는 것이 바람직합니다.
🔍 [통합 검색] 쿼리: '임대차계약증서에 명시된 증액 비율을 초과하여 월세를 인상하려는 경우, 임차인은 임대인의 결정에 무조건 따르지 않아도 됩니다. 또한, 계약 기간 내에서 월세를 인상하는 횟수에 대한 규정은 지역 법률과 계약서 내용에 따라 다를 수 있으나, 일반적으로 1년에 한 번만 인상할 수 있다는 것은 상한선일 수 있습니다. 구체적인 조항 확인을 위해 임대차계약증서를 참조하거나 법률 전문가의 조언을 구하는 것이 바람직합니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.9978] [P-1.0] 제10조 초과 차임 반환 청구
 - [Score: 0.9531] [P-1.0] 제7조 차임 증감 청구권
 - [Score: 0.9264] [P-2.0] 제8조 차임 증액 제한 기준
 - [Score: 0.8890] [P-1.0] 제7조 월차임 전환 시 산정률 제한
 - [Score: 0.8478] [P-9.0]  - 판결요지
 - [Score: 0.8344] [P-9.0]  - 이유_p1
 - [Score: 0.6999] [P-1.0] 제6조 계약갱신 요구권 제한 및 손해배상 규정
 - [Score: 0.5536] [P-6.0] 제18조 임대보증금과 월임대료 전환 규정
 - [Score: 0.4497] [P-7.0] 제3조 주택임대차분쟁조정위원회 설치 및 운영
 - [Score: 0.4358] [P-7.0] 제2조 경기도 주택임대차분쟁조정위원회 설치 및 

In [37]:
test_query = "보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임차보증금을 감액하여 월세로 변경하려는 경우, 월세액수에 대한 상한 기준이 있나요?
🔍 [통합 검색] 쿼리: '임차보증금을 감액하여 월세로 변경하려는 경우, 월세액수에 대한 상한 기준이 있나요?'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.8789] [P-6.0] 제18조 임대보증금과 월임대료 전환 규정
 - [Score: 0.7980] [P-1.0] 제7조 월차임 전환 시 산정률 제한
 - [Score: 0.3083] [P-9.0]  - 이유_p5
🤖 답변 생성 중...
한국에서 임차인이 임차보증금을 감액하여 월세로 전환하려는 경우, 구체적인 월세액수에 대한 상한 기준은 법률이나 정부 지침에 명시되어 있지 않습니다. 그러나 일반적으로 다음과 같은 고려사항들이 있습니다:

1. **지역별 차이**: 각 지자체의 주택정책이나 지역 특성에 따라 월세 상한선이 다르게 적용될 수 있습니다. 일부 지자체에서는 월세 상한선을 설정하고 있지만, 이는 지역마다 상이할 수 있습니다.

2. **법률 및 지침**:
   - **주택임대차보호법**은 주로 임대차 계약의 기본 조건과 임차인의 권리 보호에 초점을 맞추고 있으며, 월세 상한에 대한 직접적인 규정은 없습니다.
   - **지자체 조례**: 일부 지자체에서는 주택임대차 관련 조례를 통해 월세 상한선을 명시하고 있을 수 있습니다. 예를 들어, 서울시나 부산시 등에서는 월세 상한에 대한 가이드라인을 제공할 수 있습니다.

3. **실제 적용 사례**:
   - 실제 부동산 거래나 계약에서는 임대인과 임차인 간의 합의에 따라 월세액수가 결정되며, 이 과정에서 시장 상황, 지역 경제 상황 등 다양한 요인이 고려됩니다.
   - 만약 월세 상한에 대한 명확한 정보가 필요하다면, 해당 지역의 지자체 주택과나 법률 전문가와 상담하는 것이 좋습니다.

결론적으로, 월세액수에 대한 구체적인 상한 기준은 지역에 따라 다르며, 공식적인 

In [38]:
test_query = "집이 경매로 넘어간다고 들었어요. 저는 보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임대인이 경매로 집을 처분한다고 들었습니다. 보증금이 크지 않아 일부라도 우선 변제받을 수 있을까요? 지역별로 기준이 어떻게 다른지 궁금합니다.
🔍 [통합 검색] 쿼리: '임대인이 경매로 집을 처분한다고 들었습니다. 보증금이 크지 않아 일부라도 우선 변제받을 수 있을까요? 지역별로 기준이 어떻게 다른지 궁금합니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.8412] [P-1.0] 제24조 지방세 우선 징수 특례
 - [Score: 0.8387] [P-1.0] 제8조 보증금 우선 변제 보호 규정
 - [Score: 0.7048] [P-1.0] 제3조 보증금 회수 및 우선변제 권리 관련 조문 제목:

- 보증금 회수 절차
- 임차인의 우선변제권 행사
- 보증금 변제 우선권
- 보증금 회수 및 이의신청 절차
- 금융기관의 보증금 변제 범위
 - [Score: 0.6394] [P-1.0] 제3조 경매에 의한 임차권 소멸
 - [Score: 0.5159] [P-9.0]  - 이유
🤖 답변 생성 중...
임대인이 경매로 집을 처분하는 상황에서 보증금 우선 변제 가능성과 지역별 기준에 대해 설명드리겠습니다. 한국의 경우, 주로 민사집행법과 관련 법률에 따라 이러한 상황이 처리됩니다.

### 보증금 우선 변제 가능성

1. **일반적인 경우**:
   - **경매 진행 시**: 임대인이 경매로 집을 처분하더라도, 일반적으로 임차인의 보증금은 최우선으로 변제됩니다. 즉, 경매 수익금에서 보증금이 먼저 지급되는 것이 원칙입니다.
   - **경매 수익금 분배**: 경매 수익금은 다음과 같이 분배됩니다:
     - **보증금**: 최우선으로 변제됩니다.
     - **임대료 미지급액**: 보증금을 초과하는 금액 중 미지급 임대료가 변제됩니다.
     - **기타 채무**: 남은 금액으로 임대인의 다른 채무가 변제됩니다.

2. **보증금 규모에 따른 차이**:
  

In [39]:
test_query = "재계약하면서 월세를 많이 올렸는데, 나중에 보니까 법에서 정한 비율보다 더 낸 것 같아요. 이미 낸 돈도 돌려달라고 할 수 있나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 재계약 시 차임을 증액했으나 법에서 정한 상한선인 5퍼센트를 초과하여 차임을 받았다면, 이미 납부한 금액에 대해 반환을 청구할 수 있습니다. 특히, 이미 납부한 금액은 우선변제권이 인정되어 반환을 요구할 수 있습니다.
🔍 [통합 검색] 쿼리: '재계약 시 차임을 증액했으나 법에서 정한 상한선인 5퍼센트를 초과하여 차임을 받았다면, 이미 납부한 금액에 대해 반환을 청구할 수 있습니다. 특히, 이미 납부한 금액은 우선변제권이 인정되어 반환을 요구할 수 있습니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.7644] [P-9.0]  - 이유_p2
 - [Score: 0.5912] [P-1.0] 제10조 초과 차임 반환 청구
 - [Score: 0.5081] [P-1.0] 제7조 차임 증감 청구권
 - [Score: 0.4020] [P-2.0] 제8조 차임 증액 제한 기준
 - [Score: 0.3783] [P-9.0]  - 이유_p1
 - [Score: 0.3022] [P-4.0] 제628조 차임증감청구권
🤖 답변 생성 중...
재계약 시 차임을 증액하면서 법에서 정한 상한선인 5%를 초과하여 차임을 부과한 경우, 다음과 같은 절차와 고려사항들이 중요합니다:

### 1. **법적 근거 확인**
   - **관련 법률**: 한국에서는 주로 「민법」과 「임대차보호법」이 관련되어 있습니다. 특히, 임대차보호법 제6조는 차임의 증액에 대한 상한선을 명시하고 있습니다.
   - **상한선 준수**: 법에서 정한 5% 이내로 차임을 증액해야 합니다. 초과 증액은 불법으로 간주될 수 있습니다.

### 2. **이미 납부한 차임의 처리**
   - **반환 청구 가능성**: 이미 차임을 초과하여 납부한 금액에 대해서는 원칙적으로 반환 청구가 가능합니다. 이는 「민법」 제399조와 「임대차보호법」 제6조 등에서 근거를 찾을 수 있습니다.
   - **우선변제권**: 이미 

In [40]:
test_query = "전세계약은 했는데 등기는 안 했거든요. 그래서 주택임대차보호법은 적용 안 된다고 하는데 맞아요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 전세 계약은 체결했으나 등기가 이루어지지 않았습니다. 이 경우, 주택임대차보호법의 적용 여부에 대해 의문이 있으신데, 일반적으로 등기가 이루어지지 않은 상태에서는 해당 법률의 보호를 완전히 받기 어렵습니다. 정확한 적용 여부는 상황에 따라 다를 수 있으니 추가 확인이 필요할 수 있습니다.
🔍 [통합 검색] 쿼리: '전세 계약은 체결했으나 등기가 이루어지지 않았습니다. 이 경우, 주택임대차보호법의 적용 여부에 대해 의문이 있으신데, 일반적으로 등기가 이루어지지 않은 상태에서는 해당 법률의 보호를 완전히 받기 어렵습니다. 정확한 적용 여부는 상황에 따라 다를 수 있으니 추가 확인이 필요할 수 있습니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.9986] [P-1.0] 제12조 미등기 전세 계약 관련 준용
 - [Score: 0.9884] [P-1.0] 주택임대차보호법 주택임대차보호법
 - [Score: 0.9795] [P-9.0]  - 판결요지
 - [Score: 0.9702] [P-3.0] 제6조 주택 임대차 신고 절차
 - [Score: 0.9679] [P-6.0] 제2조 임차권등기명령신청서 기재사항
 - [Score: 0.9639] [P-6.0] 제2조 임대인 송달 미완료 임차권등기명령 적용
 - [Score: 0.9530] [P-1.0] 제3조 **대항력 관련 조항**
 - [Score: 0.9342] [P-6.0] 제130조 임차권설정등기 신청 절차
 - [Score: 0.9127] [P-1.0] 제6조 주택 임대차 계약 신고 의무화
 - [Score: 0.9026] [P-1.0] 제3조 주택임대차 등기 효력 관련 규정
 - [Score: 0.8933] [P-6.0] 제9조 임차권등기명령의 대위신청
 - [Score: 0.8374] [P-1.0] 제3조 임차권등기명령 관련 핵심 제목:

**임차권등기명령 신청 및 절차**
 - [Score: 0.8

In [41]:
test_query = "보증금이 크진 않은데, 소송하면 너무 복잡하고 오래 걸릴까 봐 걱정돼요. 간단하게 할 수 있는 방법은 없나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 보증금이 크지 않아 소송이 복잡하고 오래 걸릴까 봐 걱정되시군요. 간단하게 해결할 수 있는 방법으로는 **우선변제권**을 활용하거나 **대항력**을 확보하는 것이 있습니다. 또한 **조정위**를 통해 분쟁을 신속하게 해결해볼 수 있습니다. 법원 소송보다는 이러한 절차들이 비교적 빠르고 효율적일 수 있습니다.
🔍 [통합 검색] 쿼리: '보증금이 크지 않아 소송이 복잡하고 오래 걸릴까 봐 걱정되시군요. 간단하게 해결할 수 있는 방법으로는 **우선변제권**을 활용하거나 **대항력**을 확보하는 것이 있습니다. 또한 **조정위**를 통해 분쟁을 신속하게 해결해볼 수 있습니다. 법원 소송보다는 이러한 절차들이 비교적 빠르고 효율적일 수 있습니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
죄송합니다. 관련 법령이나 판례를 찾을 수 없습니다.


In [42]:
test_query = "집주인이랑 월세랑 보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 집주인이 중개사 수수료를 저보고 다 내라고 하는데요."
print(generate_final_answer(test_query))

🔄 표준화된 질문: 임차인과 임대인 사이의 월세 및 보증금 관련 분쟁으로 갈등이 지속되고 있으며, 법원 소송 대신 중재를 통해 해결하려는 노력이 없어 답답합니다. 또한 임대인이 중개수수료를 임차인에게 부담시키려는 상황입니다.
🔍 [통합 검색] 쿼리: '임차인과 임대인 사이의 월세 및 보증금 관련 분쟁으로 갈등이 지속되고 있으며, 법원 소송 대신 중재를 통해 해결하려는 노력이 없어 답답합니다. 또한 임대인이 중개수수료를 임차인에게 부담시키려는 상황입니다.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
 - [Score: 0.9734] [P-7.0] 제2조 경기도 주택임대차분쟁조정위원회 설치 및 운영
 - [Score: 0.9694] [P-1.0] 제14조 주택임대차분쟁조정위원회 설치 및 운영
 - [Score: 0.9533] [P-7.0] 제3조 주택임대차분쟁조정위원회 설치 및 운영
 - [Score: 0.9490] [P-9.0]  - 이유_p1
 - [Score: 0.9320] [P-6.0] 제18조 임대보증금과 월임대료 전환 규정
 - [Score: 0.9131] [P-1.0] 제7조 차임 증감 청구권
 - [Score: 0.8710] [P-1.0] 제3조 보증금 회수 및 우선변제 권리 관련 조문 제목:

- 보증금 회수 절차
- 임차인의 우선변제권 행사
- 보증금 변제 우선권
- 보증금 회수 및 이의신청 절차
- 금융기관의 보증금 변제 범위
 - [Score: 0.8567] [P-1.0] 제6조 계약갱신 요구권 제한 및 손해배상 규정
 - [Score: 0.8252] [P-1.0] 제3조 임차권등기명령 관련 핵심 제목:

**임차권등기명령 신청 및 절차**
 - [Score: 0.8245] [P-7.0] 제12조 주택임대차분쟁 조정 신청 절차
 - [Score: 0.8139] [P-6.0] 제2조 임차권등기명령신청서 기재사항
 - [Score: 0.7825] [P-1.0] 제21조 주택임대차분쟁 

In [43]:
test_query = "제 사건은 금액도 크지 않은데, 이런 것도 빨리 처리해 주는 방식이 있나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 제 사건 처리 속도가 빠른 방식이 있나요? 특히 금액 규모가 크지 않은 상황에서요.
🔍 [통합 검색] 쿼리: '제 사건 처리 속도가 빠른 방식이 있나요? 특히 금액 규모가 크지 않은 상황에서요.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
죄송합니다. 관련 법령이나 판례를 찾을 수 없습니다.


In [44]:
test_query = "조정안이 나왔는데 바로 답 안 하면 어떻게 돼요? 가만히 있으면 자동으로 되는 건가요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 조정안에 대한 답변을 지연하면 어떻게 될까요? 가만히 있어도 자동으로 처리되는 건가요?
🔍 [통합 검색] 쿼리: '조정안에 대한 답변을 지연하면 어떻게 될까요? 가만히 있어도 자동으로 처리되는 건가요?'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
죄송합니다. 관련 법령이나 판례를 찾을 수 없습니다.


In [45]:
test_query = "조정위원이랑 통화했는데, 뭔가 부당하게 처리된 것 같아요. 이런 사람들도 공무원처럼 책임을 지나요?"
print(generate_final_answer(test_query))

🔄 표준화된 질문: 조정위원회와 통화하셨는데 부당하게 처리된 느낌이 드시나요? 공무원처럼 해당 사안에 대해 책임을 지는지 궁금하시군요.
🔍 [통합 검색] 쿼리: '조정위원회와 통화하셨는데 부당하게 처리된 느낌이 드시나요? 공무원처럼 해당 사안에 대해 책임을 지는지 궁금하시군요.'
📊 Rerank 결과 (총 14개 중 선별):
📊 Rerank 관련도 점수 (Threshold 0.2):
죄송합니다. 관련 법령이나 판례를 찾을 수 없습니다.
